#Converting the OOT sequences to the same format as used by VOT

##Which ground-truth format is used by OOT?
This question may be answered by simply opening a sample ground-truth file and reading a sample line from it.

In [1]:
sampleGTPath = '/home/datasets/datasets1/oot-benchmark/unzipped/MountainBike/groundtruth_rect.txt'
with open(sampleGTPath, 'r') as sampleGTFile:
    sampleLines = sampleGTFile.readlines()

In [2]:
print sampleLines[0]

319,185,67,56



The OOT format seems to be $x_{0}, y_{0},w,h$.

###Exercise
Write a function to generate a list of integers from a groundtruth line.
###Solution
As the field separator may vary, we convert commas to spaces. We also strip for trailing newline characters.

In [4]:
def read_box(line):
    box = map(int, line.strip().replace(',', ' ').split())
    return box

###Exercise
Write a function to convert from boxes $x_{0}, y_{0},w,h$ to polygons $x_{0},y_{0},x_{1},y_{1},x_{2},y_{2},x_{3},y_{3}$.
###Solution
The first coordinates remain the same and the next ones are generated by increasing first the width and then the height.

In [5]:
def box_to_polygon(box):
    x0 = box[0]
    y0 = box[1]
    w = box[2]
    h = box[3]
    polygon = [x0,y0,x0+w,y0,x0+w,y0+h,x0,y0+h]
    return polygon

In [6]:
box_to_polygon(read_box(sampleLines[0]))

[319, 185, 386, 185, 386, 241, 319, 241]

###Exercise
Write a function to generate string representations of polygons.
###Solution
The default separator will be space/comma and newlines will be appended at the end. 

In [7]:
def write_polygon(polygon):
    line = ','.join(map(str, polygon)) + '\n'
    return line

###Exercise
Write a function to generate a VOT formatted ground-truth file from a OOT one.
###Solution
Input file will be read by lines and stored in memory. Each line will be converted to boxes, which will then be converted to polygons. The memory list of polygons will be written to the output file.

In [8]:
def oot_to_vot(ootPath, votPath):
    with open(ootPath, 'r') as ootGt:
        ootLines = ootGt.readlines()
    with open(votPath, 'w') as votGt:
        for line in ootLines:
            box = read_box(line)
            polygon = box_to_polygon(box)
            votGt.write(write_polygon(polygon))

###Quiz
Write a script that generates the whole VOT structure from a OOT structure.

In [9]:
import os

In [14]:
OOT_DIR = '/home/datasets/datasets1/oot-benchmark/'
unzippedDir = os.path.join(OOT_DIR, 'unzipped')
imgSuffix = 'img'
imgExtension = '.jpg'
ootGtName = 'groundtruth_rect.txt'
votGtName = 'groundtruth.txt'
sequencesDir = os.path.join(OOT_DIR, 'sequences')
if not os.path.exists(sequencesDir):
    os.mkdir(sequencesDir)
sequenceNames = os.listdir(unzippedDir)
for sequence in sequenceNames:
    sequenceDir = os.path.join(sequencesDir, sequence)
    votGtPath = os.path.join(sequencesDir, votGtName)
    ootSequenceDir = os.path.join(unzippedDir, sequence)
    ootGtPath = os.path.join(ootSequenceDir, ootGtName)
    if os.path.exists(ootGtPath):
        if not os.path.exists(sequenceDir):
            os.mkdir(sequenceDir)
        ootImageDir = os.path.join(ootSequenceDir, imgSuffix)
        for imageName in os.listdir(ootImageDir):
            if imageName.endswith(imgExtension):
                dstPath = os.path.join(sequenceDir, '{:08d}{}'.format(int(imageName[:-len(imgExtension)]), imgExtension))
                if os.path.exists(dstPath):
                    os.remove(dstPath)
                os.symlink(os.path.join(ootImageDir, imageName), dstPath)
            else:
                print 'Ommitting {}'.format(imageName)
    else:
        print 'File missing: {}'.format(ootGtPath)

Ommitting Picasa.ini
Ommitting Picasa.ini
File missing: /home/datasets/datasets1/oot-benchmark/unzipped/Jogging/groundtruth_rect.txt
